In [1]:
# what kind of files do we have?
from glob import glob
from pathlib import Path
from collections import Counter

GLOB = '/kaggle/input/chessman-image-dataset/Chessman-image-dataset/Chess/*/*'
suffixes = Counter([Path(input_file).suffix for input_file in glob(pathname=GLOB)])

print(suffixes)

Counter({'.jpg': 465, '.png': 69, '.JPG': 10, '.jpeg': 7, '.fcgi': 2, '.php': 1, '.webp': 1, '.gif': 1})


We have some files that are not images and we need to be sure to ignore them.

In [2]:
from PIL import Image

this_file = '/kaggle/input/chessman-image-dataset/Chessman-image-dataset/Chess/Bishop/00000000.JPG'
data = Image.open(this_file).convert('RGB')


In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms

device = torch.device('cpu')
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to(device=device)
model.eval()

layer = model._modules.get('avgpool')

layer_output_size = 512

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

item = normalize(to_tensor(scaler(data)))
items = list(item)
images = torch.stack(items).to(device)
# result = model(images)
# result = torch.zeros(len(images[0]), layer_output_size, 1, 1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 101MB/s]
